In [1]:
#!pip install ipypb

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from pandas import json_normalize

pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 주피터 노트북 과부하가 걸릴 수 있으므로 다시 돌려놓음
# pd.options.display.max_rows = 10

In [3]:
df = pd.read_csv('C:/Users/pc/Desktop/SPARTA/01. 프로젝트/3. 심화 프로젝트/01. 데이터/train.csv/train.csv', converters={column: json.loads for column in ['device', 'geoNetwork','totals', 'trafficSource']}, dtype={'fullVisitorId': 'str'})

In [4]:
df = df.sample(n=50000)

In [5]:
df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
549478,Referral,20170726,"{'browser': 'Chrome', 'browserVersion': 'not a...",9334152386480217693,"{'continent': 'Europe', 'subContinent': 'North...",9334152386480217693_1501087378,Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'referralPath': '/', 'campaign': '(not set)',...",1501087378,2,1501087378
615817,Social,20161121,"{'browser': 'Chrome', 'browserVersion': 'not a...",3103237221107568184,"{'continent': 'Asia', 'subContinent': 'Southea...",3103237221107568184_1479749063,Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'referralPath': '/yt/about/', 'campaign': '(n...",1479749063,1,1479749063
552582,Organic Search,20170323,"{'browser': 'Chrome', 'browserVersion': 'not a...",662103717244628710,"{'continent': 'Europe', 'subContinent': 'Weste...",662103717244628710_1490262526,Not Socially Engaged,"{'visits': '1', 'hits': '9', 'pageviews': '7',...","{'campaign': '(not set)', 'source': 'google', ...",1490262526,1,1490262526
137938,Social,20170102,"{'browser': 'Chrome', 'browserVersion': 'not a...",4449116109152542076,"{'continent': 'Americas', 'subContinent': 'Sou...",4449116109152542076_1483407074,Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'referralPath': '/yt/about/pt-BR/', 'campaign...",1483407074,1,1483407074
526942,Social,20161016,"{'browser': 'Safari', 'browserVersion': 'not a...",7942847711749099931,"{'continent': 'Americas', 'subContinent': 'Cen...",7942847711749099931_1476623894,Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'referralPath': '/yt/about/', 'campaign': '(n...",1476623894,1,1476623894


In [6]:
df = df.reset_index()

In [7]:
json_columns = ['device', 'geoNetwork','totals', 'trafficSource']

def get_columns(columns_list: list):
   df2 = df.copy()
   for column in json_columns:
        column_as_df = json_normalize(df2[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df2 = df2.drop(column, axis=1)
        df2 = pd.concat([df2, column_as_df], axis = 1)
   return df2

In [8]:
df3 = get_columns(json_columns)

# 분석용 데이터 ===============================================

### 변수설명
1. channelGrouping
2. fullVisitorId: 아이디 고유값 
5. visitNumber: 사용자의 세션 숫자, 첫번째 세션이면 1
6. visitStartTime: 타임스탬프           # 질문: 머무르는 시간인가? 아니면 접속 시간인가?
7. device_isMobile: True/False
8. device_deviceCategory: desktop/mobile/tablet
9. geoNetwork_continent: 대륙
10. geoNetwork_country: 국가
11. totals_visits: 1/Null, 세션이 1이면 이벤트 0, 세션이 null이면 어떠한 이벤트x  
12. totals_hits: 세션 내에 히트의 숫자
13. totals_pageviews: 세션 내에 페이지뷰 숫자	
14. totals_bounces: 1/0 , 1이면 아무것도 안하고 이탈
15. totals_newVisits: 첫번째 방문이면 1, 아니면 null
16. totals_transactionRevenue: 거래비용, 나누기 10^6할 것
17. trafficSource_campaign: 캠페인 트래킹 파라미터
18. trafficSource_source: 트레픽 소스의 소스로 검색엔진, 참고호스트네임, 소스url 파라미터
19. trafficSource_medium: 트레픽 소스의 매개체? 
20. trafficSource_keyword: 트래픽 소스의 키워드 
21. trafficSource_isTrueDirect: True/Null, 웹사이트에 직접 작성하였거나 북마크하면 True, 그 외에는 null
22. trafficSource_adwordsClickInfo.slot: 광고 포지션?
23. trafficSource_adwordsClickInfo.adNetworkType: “Google Search", "Content", "Search partners", "Ad Exchange", "Yahoo Japan Search", "Yahoo Japan AFS", “unknown”, 네트워크 타입

### 참고자료
- GA에 대한 자세한 설명: https://analyticsmarketing.co.kr/digital-analytics/google-analytics-basics/2430/

In [9]:
df3 = df3[['channelGrouping', 'fullVisitorId','visitNumber','visitStartTime', 'device_isMobile', 'device_deviceCategory', 'geoNetwork_continent', 'geoNetwork_country',	'totals_visits', 'totals_hits',	'totals_pageviews',	'totals_newVisits','totals_transactionRevenue','trafficSource_campaign','trafficSource_source', 'trafficSource_medium', 'trafficSource_keyword', 'trafficSource_isTrueDirect',	'trafficSource_adwordsClickInfo.slot','trafficSource_adwordsClickInfo.adNetworkType']]
df3.head(10)

,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_visits,totals_hits,totals_pageviews,totals_newVisits,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.adNetworkType
0,Referral,9334152386480217693,2,1501087378,False,desktop,Europe,United Kingdom,1,1,1,NaN,NaN,(not set),mall.googleplex.com,referral,NaN,True,NaN,NaN
1,Social,3103237221107568184,1,1479749063,False,desktop,Asia,Thailand,1,2,2,1,NaN,(not set),youtube.com,referral,NaN,NaN,NaN,NaN
2,Organic Search,662103717244628710,1,1490262526,True,tablet,Europe,Switzerland,1,9,7,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
3,Social,4449116109152542076,1,1483407074,True,mobile,Americas,Brazil,1,2,2,1,NaN,(not set),youtube.com,referral,NaN,NaN,NaN,NaN
4,Social,7942847711749099931,1,1476623894,False,desktop,Americas,Mexico,1,1,1,1,NaN,(not set),youtube.com,referral,NaN,NaN,NaN,NaN
5,Organic Search,8148557342120636352,1,1490030130,False,desktop,Europe,Slovakia,1,1,1,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN
6,Social,6919216930440358809,1,1485123231,False,desktop,Americas,United States,1,3,3,1,NaN,(not set),youtube.com,referral,NaN,NaN,NaN,NaN
7,Organic Search,1815789136636497177,1,1481157253,False,desktop,Americas,United States,1,3,3,1,NaN,(not set),ask,organic,GOOGLE ELECTRONICS,NaN,NaN,NaN
8,Referral,0328080880890336475,2,1485213283,False,desktop,Americas,United States,1,12,9,NaN,NaN,(not set),mall.googleplex.com,referral,NaN,True,NaN,NaN
9,Social,5733849451681915634,1,1477920263,False,desktop,Europe,Ukraine,1,2,2,1,NaN,(not set),youtube.com,referral,NaN,NaN,NaN,NaN


### 01. 데이터 타입변환

In [10]:
# 문자열로
df3['fullVisitorId']=df3['fullVisitorId'].astype(str)
# 날짜로
df3['visitStartTime']=pd.to_datetime(df3['visitStartTime'],unit='s')
# 숫자로(float -> int변환이 안돼서 일단 그대로 둠)
df3['totals_visits']=pd.to_numeric(df3['totals_visits'])
df3['totals_hits']=pd.to_numeric(df3['totals_hits'])
df3['totals_pageviews']=pd.to_numeric(df3['totals_pageviews'])
df3['totals_newVisits']=pd.to_numeric(df3['totals_newVisits'])
df3['totals_transactionRevenue']=pd.to_numeric(df3['totals_transactionRevenue'])


In [11]:
# 시간 제외 날짜만 visitStartTime2
df3['visitStartTime2']=df3['visitStartTime'].dt.strftime("%y-%m-%d")
df3['visitStartTime2']=pd.to_datetime(df3['visitStartTime2'],format='%y-%m-%d')

### 02. 인코딩

In [12]:
# trafficSource_campaign 인코딩
def get_cam(x):
    if x == '(not set)':
        return 0
    else:
        return 1

df3['trafficSource_campaign_en'] = df3['trafficSource_campaign'].apply(get_cam)

# totals_transactionRevenue 인코딩(nan -> 0)
df3['totals_transactionRevenue'].fillna(0, inplace=True)
df3['totals_transactionRevenue']=np.power(df3['totals_transactionRevenue'],1/6).round(2)

# null -> 0으로
df3['totals_newVisits'].fillna(0, inplace=True)

In [46]:
df3

,fullVisitorId,channelGrouping,visitNumber,visitStartTime,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_visits,totals_hits,totals_pageviews,totals_newVisits,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.adNetworkType,visitStartTime2,trafficSource_campaign_en,visitCycle,First_visit_month
0,9334152386480217693,Referral,2,2017-07-26 16:42:58,False,desktop,Europe,United Kingdom,1,1,1.0,0.0,0.0,(not set),mall.googleplex.com,referral,NaN,True,NaN,NaN,2017-07-26,0,2017.07,2017.07
1,3103237221107568184,Social,1,2016-11-21 17:24:23,False,desktop,Asia,Thailand,1,2,2.0,1.0,0.0,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,2016-11-21,0,2016.11,2016.11
2,662103717244628710,Organic Search,1,2017-03-23 09:48:46,True,tablet,Europe,Switzerland,1,9,7.0,1.0,0.0,(not set),google,organic,(not provided),NaN,NaN,NaN,2017-03-23,0,2017.03,2017.03
3,4449116109152542076,Social,1,2017-01-03 01:31:14,True,mobile,Americas,Brazil,1,2,2.0,1.0,0.0,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,2017-01-03,0,2017.01,2017.01
4,7942847711749099931,Social,1,2016-10-16 13:18:14,False,desktop,Americas,Mexico,1,1,1.0,1.0,0.0,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,2016-10-16,0,2016.10,2016.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1736695160396777911,Organic Search,1,2016-11-16 22:13:18,False,desktop,Americas,United States,1,7,7.0,1.0,0.0,(not set),google,organic,(not provided),NaN,NaN,NaN,2016-11-16,0,2016.11,2016.11
49996,6589579712030812313,Referral,5,2017-01-03 21:55:13,False,desktop,Americas,United States,1,1,1.0,0.0,0.0,(not set),sites.google.com,referral,NaN,True,NaN,NaN,2017-01-03,0,2017.01,2017.01
49997,5098967189493204018,Organic Search,1,2017-04-14 03:03:54,True,mobile,Americas,United States,1,1,1.0,1.0,0.0,(not set),google,organic,(not provided),NaN,NaN,NaN,2017-04-14,0,2017.04,2017.04
49998,0970171844996020850,Paid Search,1,2017-08-01 01:09:14,False,desktop,Americas,United States,1,4,4.0,1.0,0.0,AW - Dynamic Search Ads Whole Site,google,cpc,6qEhsCssdK0z36ri,NaN,Top,Google Search,2017-08-01,1,2017.08,2017.08


In [14]:
df3.nunique()

channelGrouping                                     8
fullVisitorId                                   48321
visitNumber                                       163
visitStartTime                                  49947
device_isMobile                                     2
device_deviceCategory                               3
geoNetwork_continent                                6
geoNetwork_country                                175
totals_visits                                       1
totals_hits                                       144
totals_pageviews                                  111
totals_newVisits                                    2
totals_transactionRevenue                         368
trafficSource_campaign                              8
trafficSource_source                              128
trafficSource_medium                                7
trafficSource_keyword                             404
trafficSource_isTrueDirect                          1
trafficSource_adwordsClickIn

In [15]:
#df3.to_csv('./result.csv')

### 03-1. 코호트분석(전체대상)

https://data-marketing-bk.tistory.com/entry/Python-COHORT-%EB%B6%84%EC%84%9D-%EA%B0%9C%EB%85%90%EB%B6%80%ED%84%B0-%EC%8B%A4%EC%A0%84-%EC%BD%94%EB%93%9C%EA%B9%8C%EC%A7%80-feat-%EB%B6%84%EC%84%9D-%EC%98%88%EC%A0%9C-%ED%8F%AC%ED%95%A8

In [36]:
df3['visitCycle'] = df3['visitStartTime2'].dt.strftime('%Y.%m')

In [37]:
first_visit = df3.groupby('fullVisitorId')['visitStartTime2'].min()
first_visit.sort_values()
first_visit.count()


48321

In [38]:
df3 = df3.set_index('fullVisitorId')
df3['First_visit_month'] = first_visit.dt.strftime('%Y.%m')
df3.reset_index(inplace=True)

In [39]:
co1 = df3.groupby(['First_visit_month', 'visitCycle'])['fullVisitorId'].nunique()
co1 = co1.reset_index()
co1

,First_visit_month,visitCycle,fullVisitorId
0,2016.08,2016.08,4135
1,2016.08,2016.09,49
2,2016.08,2016.10,16
3,2016.08,2016.11,13
4,2016.08,2016.12,8
...,...,...,...
75,2017.06,2017.07,28
76,2017.06,2017.08,1
77,2017.07,2017.07,3871
78,2017.07,2017.08,3


In [40]:
from ipypb import ipb
temp1=[]

for i in ipb(range(co1.shape[0])):
    f_first_visit = pd.to_datetime(co1.First_visit_month[i]).to_period('M')
    f_visit_cycle = pd.to_datetime(co1.visitCycle[i]).to_period('M')
    
    month_diff = (f_visit_cycle - f_first_visit).n
    temp1.append(month_diff)

co1['CohortPeriod'] = temp1

 [████████████████████████████████████████████████████████████] 80/80 [00:01<00:00, 0.01s/it]

In [41]:
co1.sample(5)

,First_visit_month,visitCycle,fullVisitorId,CohortPeriod
49,2017.01,2017.01,3424,0
15,2016.09,2017.02,4,5
34,2016.11,2017.02,9,3
37,2016.11,2017.05,3,6
77,2017.07,2017.07,3871,0


In [42]:
co_retention1 = co1.set_index(['First_visit_month', 'CohortPeriod'])
co_retention1 = co_retention1['fullVisitorId'].unstack(1)
retention1 = co_retention1.div(co_retention1[0], axis=0)

In [43]:
retention1 = retention1 * 100
retention1

CohortPeriod,0,1,2,3,4,5,6,7,8,9,10
First_visit_month,,,,,,,,,,,
2016.08,100.0,1.185006,0.386941,0.314389,0.193470,0.169287,0.072551,0.169287,0.072551,0.024184,NaN
2016.09,100.0,1.036958,0.292475,0.319064,0.186121,0.106355,0.079766,0.079766,0.079766,0.026589,0.026589
2016.10,100.0,0.779575,0.311830,0.233873,0.058468,0.058468,0.038979,0.097447,0.077958,0.019489,NaN
2016.11,100.0,0.540452,0.229283,0.147396,0.114641,0.163773,0.049132,0.016377,0.114641,0.016377,NaN
2016.12,100.0,0.715605,0.253924,0.161588,0.138504,0.069252,0.046168,0.069252,NaN,NaN,NaN
2017.01,100.0,0.671729,0.292056,0.233645,0.058411,0.058411,NaN,NaN,NaN,NaN,NaN
2017.02,100.0,0.477185,0.328064,0.238592,0.089472,0.208768,0.059648,NaN,NaN,NaN,NaN
2017.03,100.0,0.855615,0.240642,0.053476,0.106952,NaN,NaN,NaN,NaN,NaN,NaN
2017.04,100.0,0.646612,0.281136,0.112454,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 03-2. 코호트분석(물품구매자 대상)

In [24]:
df4 = df3[df3['totals_transactionRevenue'] != 0]
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 625 entries, 10 to 49686
Data columns (total 24 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fullVisitorId                                 625 non-null    object        
 1   channelGrouping                               625 non-null    object        
 2   visitNumber                                   625 non-null    int64         
 3   visitStartTime                                625 non-null    datetime64[ns]
 4   device_isMobile                               625 non-null    bool          
 5   device_deviceCategory                         625 non-null    object        
 6   geoNetwork_continent                          625 non-null    object        
 7   geoNetwork_country                            625 non-null    object        
 8   totals_visits                                 625 non-null    int64     

In [25]:
df4['visitCycle'] = df4['visitStartTime2'].dt.strftime('%Y.%m')

first_visit = df4.groupby('fullVisitorId')['visitStartTime2'].min()
first_visit.sort_values()
first_visit.count()

df4 = df4.set_index('fullVisitorId')
df4['First_visit_month'] = first_visit.dt.strftime('%Y.%m')
df4.reset_index(inplace=True)

co2 = df4.groupby(['First_visit_month', 'visitCycle'])['fullVisitorId'].nunique()
co2 = co2.reset_index()

from ipypb import ipb
temp=[]

for i in ipb(range(co2.shape[0])):
    f_first_visit = pd.to_datetime(co2.First_visit_month[i]).to_period('M')
    f_visit_cycle = pd.to_datetime(co2.visitCycle[i]).to_period('M')
    
    month_diff = (f_visit_cycle - f_first_visit).n
    temp.append(month_diff)

co2['CohortPeriod'] = temp

co_retention2 = co2.set_index(['First_visit_month', 'CohortPeriod'])
co_retention2 = co_retention2['fullVisitorId'].unstack(1)
retention2 = co_retention2.div(co_retention2[0], axis=0)

retention2 = retention2 * 100
retention2

C:\Users\pc\AppData\Local\Temp\ipykernel_23016\2740662656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['visitCycle'] = df4['visitStartTime2'].dt.strftime('%Y.%m')


 [████████████████████████████████████████████████████████████] 17/17 [00:00<00:00, 0.00s/it]

CohortPeriod,0,1,2,5
First_visit_month,,,,
2016.08,100.0,1.960784,NaN,NaN
2016.09,100.0,1.785714,NaN,NaN
2016.10,100.0,NaN,NaN,NaN
2016.11,100.0,NaN,NaN,NaN
2016.12,100.0,NaN,NaN,1.06383
2017.01,100.0,NaN,2.380952,NaN
2017.02,100.0,NaN,NaN,NaN
2017.03,100.0,NaN,NaN,NaN
2017.04,100.0,NaN,NaN,NaN


### 03-3. 코호트분석(그냥 고객)

In [30]:
df5 = df3[df3['trafficSource_campaign'] == '(not set)']
df5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47844 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fullVisitorId                                 47844 non-null  object        
 1   channelGrouping                               47844 non-null  object        
 2   visitNumber                                   47844 non-null  int64         
 3   visitStartTime                                47844 non-null  datetime64[ns]
 4   device_isMobile                               47844 non-null  bool          
 5   device_deviceCategory                         47844 non-null  object        
 6   geoNetwork_continent                          47844 non-null  object        
 7   geoNetwork_country                            47844 non-null  object        
 8   totals_visits                                 47844 non-null  int64    

In [32]:
df5['visitCycle'] = df5['visitStartTime2'].dt.strftime('%Y.%m')

first_visit = df5.groupby('fullVisitorId')['visitStartTime2'].min()
first_visit.sort_values()
first_visit.count()

df5 = df5.set_index('fullVisitorId')
df5['First_visit_month'] = first_visit.dt.strftime('%Y.%m')
df5.reset_index(inplace=True)

co3 = df5.groupby(['First_visit_month', 'visitCycle'])['fullVisitorId'].nunique()
co3 = co3.reset_index()

from ipypb import ipb
temp3=[]

for i in ipb(range(co3.shape[0])):
    f_first_visit = pd.to_datetime(co3.First_visit_month[i]).to_period('M')
    f_visit_cycle = pd.to_datetime(co3.visitCycle[i]).to_period('M')
    
    month_diff = (f_visit_cycle - f_first_visit).n
    temp3.append(month_diff)

co3['CohortPeriod'] = temp3

co_retention3 = co3.set_index(['First_visit_month', 'CohortPeriod'])
co_retention3 = co_retention3['fullVisitorId'].unstack(1)
retention3 = co_retention3.div(co_retention3[0], axis=0)

retention3 = retention3 * 100
retention3

 [████████████████████████████████████████████████████████████] 80/80 [00:00<00:00, 0.00s/it]

CohortPeriod,0,1,2,3,4,5,6,7,8,9,10
First_visit_month,,,,,,,,,,,
2016.08,100.0,1.232084,0.377169,0.326880,0.201157,0.150867,0.075434,0.150867,0.075434,0.025145,NaN
2016.09,100.0,0.962861,0.275103,0.302613,0.192572,0.110041,0.055021,0.082531,0.082531,0.027510,0.02751
2016.10,100.0,0.768298,0.323494,0.202184,0.040437,0.040437,0.040437,0.060655,0.060655,0.020218,NaN
2016.11,100.0,0.555836,0.202122,0.117905,0.101061,0.168435,0.050531,0.016844,0.117905,0.016844,NaN
2016.12,100.0,0.678623,0.218129,0.145419,0.145419,0.072710,0.048473,0.072710,NaN,NaN,NaN
2017.01,100.0,0.733945,0.305810,0.244648,0.061162,0.061162,NaN,NaN,NaN,NaN,NaN
2017.02,100.0,0.467873,0.311915,0.249532,0.093575,0.218341,0.062383,NaN,NaN,NaN,NaN
2017.03,100.0,0.807799,0.250696,0.055710,0.083565,NaN,NaN,NaN,NaN,NaN,NaN
2017.04,100.0,0.584795,0.292398,0.116959,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 03-4. 코호트분석(캠페인을 통한 고객)

In [28]:
df6 = df3[df3['trafficSource_campaign'] != '(not set)']
df6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2156 entries, 18 to 49998
Data columns (total 24 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fullVisitorId                                 2156 non-null   object        
 1   channelGrouping                               2156 non-null   object        
 2   visitNumber                                   2156 non-null   int64         
 3   visitStartTime                                2156 non-null   datetime64[ns]
 4   device_isMobile                               2156 non-null   bool          
 5   device_deviceCategory                         2156 non-null   object        
 6   geoNetwork_continent                          2156 non-null   object        
 7   geoNetwork_country                            2156 non-null   object        
 8   totals_visits                                 2156 non-null   int64    

In [34]:
df6['visitCycle'] = df6['visitStartTime2'].dt.strftime('%Y.%m')

first_visit = df6.groupby('fullVisitorId')['visitStartTime2'].min()
first_visit.sort_values()
first_visit.count()

df6 = df6.set_index('fullVisitorId')
df6['First_visit_month'] = first_visit.dt.strftime('%Y.%m')
df6.reset_index(inplace=True)

co4 = df6.groupby(['First_visit_month', 'visitCycle'])['fullVisitorId'].nunique()
co4 = co4.reset_index()

from ipypb import ipb
temp4=[]

for i in ipb(range(co4.shape[0])):
    f_first_visit = pd.to_datetime(co4.First_visit_month[i]).to_period('M')
    f_visit_cycle = pd.to_datetime(co4.visitCycle[i]).to_period('M')
    
    month_diff = (f_visit_cycle - f_first_visit).n
    temp4.append(month_diff)

co4['CohortPeriod'] = temp4

co_retention4 = co4.set_index(['First_visit_month', 'CohortPeriod'])
co_retention4 = co_retention4['fullVisitorId'].unstack(1)
retention4 = co_retention4.div(co_retention4[0], axis=0)

retention4 = retention4 * 100
retention4

C:\Users\pc\AppData\Local\Temp\ipykernel_23016\719209069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['visitCycle'] = df6['visitStartTime2'].dt.strftime('%Y.%m')


 [████████████████████████████████████████████████████████████] 25/25 [00:00<00:00, 0.00s/it]

CohortPeriod,0,1,2,3,4
First_visit_month,,,,,
2016.08,100.0,NaN,NaN,NaN,NaN
2016.09,100.0,0.787402,NaN,NaN,NaN
2016.10,100.0,1.047120,NaN,1.04712,0.523560
2016.11,100.0,NaN,0.574713,NaN,NaN
2016.12,100.0,0.476190,0.476190,NaN,NaN
2017.01,100.0,NaN,NaN,NaN,NaN
2017.02,100.0,NaN,NaN,NaN,NaN
2017.03,100.0,0.636943,NaN,NaN,0.636943
2017.04,100.0,2.097902,NaN,NaN,NaN


### 전체 비교

In [45]:
print('전체=========================================')
display(retention1)
print('구매자=========================================')
display(retention2)
print('no 캠페인=========================================')
display(retention3)
print('캠페인=========================================')
display(retention4)

전체=========================================


CohortPeriod,0,1,2,3,4,5,6,7,8,9,10
First_visit_month,,,,,,,,,,,
2016.08,100.0,1.185006,0.386941,0.314389,0.193470,0.169287,0.072551,0.169287,0.072551,0.024184,NaN
2016.09,100.0,1.036958,0.292475,0.319064,0.186121,0.106355,0.079766,0.079766,0.079766,0.026589,0.026589
2016.10,100.0,0.779575,0.311830,0.233873,0.058468,0.058468,0.038979,0.097447,0.077958,0.019489,NaN
2016.11,100.0,0.540452,0.229283,0.147396,0.114641,0.163773,0.049132,0.016377,0.114641,0.016377,NaN
2016.12,100.0,0.715605,0.253924,0.161588,0.138504,0.069252,0.046168,0.069252,NaN,NaN,NaN
2017.01,100.0,0.671729,0.292056,0.233645,0.058411,0.058411,NaN,NaN,NaN,NaN,NaN
2017.02,100.0,0.477185,0.328064,0.238592,0.089472,0.208768,0.059648,NaN,NaN,NaN,NaN
2017.03,100.0,0.855615,0.240642,0.053476,0.106952,NaN,NaN,NaN,NaN,NaN,NaN
2017.04,100.0,0.646612,0.281136,0.112454,NaN,NaN,NaN,NaN,NaN,NaN,NaN


구매자=========================================


CohortPeriod,0,1,2,5
First_visit_month,,,,
2016.08,100.0,1.960784,NaN,NaN
2016.09,100.0,1.785714,NaN,NaN
2016.10,100.0,NaN,NaN,NaN
2016.11,100.0,NaN,NaN,NaN
2016.12,100.0,NaN,NaN,1.06383
2017.01,100.0,NaN,2.380952,NaN
2017.02,100.0,NaN,NaN,NaN
2017.03,100.0,NaN,NaN,NaN
2017.04,100.0,NaN,NaN,NaN


no 캠페인=========================================


CohortPeriod,0,1,2,3,4,5,6,7,8,9,10
First_visit_month,,,,,,,,,,,
2016.08,100.0,1.232084,0.377169,0.326880,0.201157,0.150867,0.075434,0.150867,0.075434,0.025145,NaN
2016.09,100.0,0.962861,0.275103,0.302613,0.192572,0.110041,0.055021,0.082531,0.082531,0.027510,0.02751
2016.10,100.0,0.768298,0.323494,0.202184,0.040437,0.040437,0.040437,0.060655,0.060655,0.020218,NaN
2016.11,100.0,0.555836,0.202122,0.117905,0.101061,0.168435,0.050531,0.016844,0.117905,0.016844,NaN
2016.12,100.0,0.678623,0.218129,0.145419,0.145419,0.072710,0.048473,0.072710,NaN,NaN,NaN
2017.01,100.0,0.733945,0.305810,0.244648,0.061162,0.061162,NaN,NaN,NaN,NaN,NaN
2017.02,100.0,0.467873,0.311915,0.249532,0.093575,0.218341,0.062383,NaN,NaN,NaN,NaN
2017.03,100.0,0.807799,0.250696,0.055710,0.083565,NaN,NaN,NaN,NaN,NaN,NaN
2017.04,100.0,0.584795,0.292398,0.116959,NaN,NaN,NaN,NaN,NaN,NaN,NaN


캠페인=========================================


CohortPeriod,0,1,2,3,4
First_visit_month,,,,,
2016.08,100.0,NaN,NaN,NaN,NaN
2016.09,100.0,0.787402,NaN,NaN,NaN
2016.10,100.0,1.047120,NaN,1.04712,0.523560
2016.11,100.0,NaN,0.574713,NaN,NaN
2016.12,100.0,0.476190,0.476190,NaN,NaN
2017.01,100.0,NaN,NaN,NaN,NaN
2017.02,100.0,NaN,NaN,NaN,NaN
2017.03,100.0,0.636943,NaN,NaN,0.636943
2017.04,100.0,2.097902,NaN,NaN,NaN
